# Data Transformation Patterns

Data transformation is the **T** in ETL/ELT pipelines—the process of converting raw data into a format suitable for analysis, reporting, or downstream applications.

## Why Transformation Matters

| Challenge | Transformation Solution |
|-----------|------------------------|
| Inconsistent formats | Standardization & normalization |
| Missing/invalid data | Cleansing & imputation |
| Disparate sources | Integration & joining |
| Raw granularity | Aggregation & summarization |
| Lacking context | Enrichment & derivation |

## Transformation Categories

```
┌─────────────────────────────────────────────────────────────┐
│                  DATA TRANSFORMATIONS                       │
├─────────────────┬─────────────────┬─────────────────────────┤
│   STRUCTURAL    │   SEMANTIC      │   ANALYTICAL            │
├─────────────────┼─────────────────┼─────────────────────────┤
│ • Type casting  │ • Cleansing     │ • Aggregation           │
│ • Renaming      │ • Normalization │ • Joining/Merging       │
│ • Reshaping     │ • Enrichment    │ • Window functions      │
│ • Filtering     │ • Standardizing │ • Derived metrics       │
└─────────────────┴─────────────────┴─────────────────────────┘
```

---
## 1. Data Cleansing

**Data cleansing** fixes or removes incorrect, corrupted, duplicate, or incomplete data.

### Common Cleansing Operations
- **Null/Missing handling**: Fill, drop, or interpolate
- **Duplicate removal**: Identify and deduplicate records
- **Outlier detection**: Cap, remove, or flag anomalies
- **Format standardization**: Consistent date/string formats

In [ ]:
import pandas as pd
import numpy as np

# Sample dirty data
raw_data = pd.DataFrame({
    'customer_id': [1, 2, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Bob', None, 'Charlie', 'Diana'],
    'email': ['alice@test.com', 'BOB@TEST.COM', 'bob@test.com', 'invalid', 'charlie@test.com', None],
    'amount': [100.0, 200.0, 200.0, -50.0, 1000000.0, 300.0],
    'date': ['2024-01-15', '01/20/2024', '2024-01-20', '2024-02-01', '2024-02-15', 'invalid']
})

print("=== Raw Data ===")
print(raw_data)
print(f"\nDuplicates: {raw_data.duplicated(subset=['customer_id']).sum()}")
print(f"Null values:\n{raw_data.isnull().sum()}")

In [ ]:
def cleanse_data(df: pd.DataFrame) -> pd.DataFrame:
    """Apply comprehensive data cleansing."""
    cleaned = df.copy()
    
    # 1. Remove duplicates (keep first occurrence)
    cleaned = cleaned.drop_duplicates(subset=['customer_id'], keep='first')
    
    # 2. Handle missing values
    cleaned['name'] = cleaned['name'].fillna('Unknown')
    cleaned['email'] = cleaned['email'].fillna('no-email@placeholder.com')
    
    # 3. Standardize email format (lowercase)
    cleaned['email'] = cleaned['email'].str.lower().str.strip()
    
    # 4. Handle outliers in amount (cap at 99th percentile)
    cap_value = cleaned['amount'].quantile(0.99)
    cleaned['amount'] = cleaned['amount'].clip(lower=0, upper=cap_value)
    
    # 5. Standardize date format
    cleaned['date'] = pd.to_datetime(cleaned['date'], errors='coerce')
    cleaned['date'] = cleaned['date'].fillna(pd.Timestamp.now())
    
    return cleaned.reset_index(drop=True)

cleaned_data = cleanse_data(raw_data)
print("=== Cleaned Data ===")
print(cleaned_data)

---
## 2. Data Normalization

**Normalization** transforms data to a common scale or format without distorting differences.

### Types of Normalization

| Type | Formula | Use Case |
|------|---------|----------|
| Min-Max | $(x - min) / (max - min)$ | Scale to [0, 1] range |
| Z-Score | $(x - \mu) / \sigma$ | Standardize distribution |
| Decimal Scaling | $x / 10^j$ | Move decimal point |
| Log Transform | $\log(x)$ | Reduce skewness |

In [ ]:
# Normalization examples
sales_data = pd.DataFrame({
    'product': ['A', 'B', 'C', 'D', 'E'],
    'revenue': [1000, 5000, 15000, 50000, 200000],
    'units_sold': [10, 50, 100, 300, 800]
})

def normalize_column(series: pd.Series, method: str = 'minmax') -> pd.Series:
    """Normalize a numeric series using various methods."""
    if method == 'minmax':
        return (series - series.min()) / (series.max() - series.min())
    elif method == 'zscore':
        return (series - series.mean()) / series.std()
    elif method == 'log':
        return np.log1p(series)  # log(1 + x) to handle zeros
    else:
        raise ValueError(f"Unknown method: {method}")

# Apply different normalizations
normalized = sales_data.copy()
normalized['revenue_minmax'] = normalize_column(sales_data['revenue'], 'minmax')
normalized['revenue_zscore'] = normalize_column(sales_data['revenue'], 'zscore')
normalized['revenue_log'] = normalize_column(sales_data['revenue'], 'log')

print("=== Normalized Data ===")
print(normalized.round(3))

---
## 3. Data Enrichment

**Enrichment** enhances data by adding context, derived fields, or external information.

### Common Enrichment Patterns
- **Derived fields**: Calculate new columns from existing ones
- **Lookups**: Join with reference/dimension tables
- **Geocoding**: Add location data from addresses
- **Time-based**: Extract date parts, add business days

In [ ]:
# Transaction data
transactions = pd.DataFrame({
    'transaction_id': range(1, 6),
    'customer_id': [101, 102, 101, 103, 102],
    'product_id': ['P001', 'P002', 'P003', 'P001', 'P002'],
    'quantity': [2, 1, 5, 3, 2],
    'unit_price': [29.99, 149.99, 9.99, 29.99, 149.99],
    'transaction_date': pd.to_datetime(['2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19'])
})

# Reference tables for enrichment
customers = pd.DataFrame({
    'customer_id': [101, 102, 103],
    'customer_name': ['Alice Corp', 'Bob LLC', 'Charlie Inc'],
    'tier': ['Gold', 'Silver', 'Bronze']
})

products = pd.DataFrame({
    'product_id': ['P001', 'P002', 'P003'],
    'product_name': ['Widget', 'Gadget', 'Accessory'],
    'category': ['Hardware', 'Electronics', 'Accessories']
})

print("=== Original Transactions ===")
print(transactions)

In [ ]:
def enrich_transactions(txn: pd.DataFrame, cust: pd.DataFrame, prod: pd.DataFrame) -> pd.DataFrame:
    """Enrich transactions with customer and product details."""
    enriched = txn.copy()
    
    # 1. Derived fields
    enriched['total_amount'] = enriched['quantity'] * enriched['unit_price']
    enriched['day_of_week'] = enriched['transaction_date'].dt.day_name()
    enriched['is_weekend'] = enriched['transaction_date'].dt.dayofweek >= 5
    enriched['fiscal_quarter'] = enriched['transaction_date'].dt.quarter
    
    # 2. Lookup enrichment - join with dimension tables
    enriched = enriched.merge(cust[['customer_id', 'customer_name', 'tier']], 
                               on='customer_id', how='left')
    enriched = enriched.merge(prod[['product_id', 'product_name', 'category']], 
                               on='product_id', how='left')
    
    # 3. Tier-based discount calculation
    tier_discounts = {'Gold': 0.15, 'Silver': 0.10, 'Bronze': 0.05}
    enriched['discount_rate'] = enriched['tier'].map(tier_discounts)
    enriched['discounted_amount'] = enriched['total_amount'] * (1 - enriched['discount_rate'])
    
    return enriched

enriched_txn = enrich_transactions(transactions, customers, products)
print("=== Enriched Transactions ===")
print(enriched_txn[['transaction_id', 'customer_name', 'product_name', 'total_amount', 'discounted_amount', 'day_of_week']])

---
## 4. Type Conversion

**Type conversion** ensures data types are appropriate for storage, computation, and analysis.

### Common Conversions

| From | To | Method |
|------|-----|--------|
| String → Date | `pd.to_datetime()` | Parse date strings |
| Float → Int | `.astype(int)` | Truncate decimals |
| Object → Category | `.astype('category')` | Memory optimization |
| String → Numeric | `pd.to_numeric()` | Parse numbers |

In [ ]:
# Data with mixed types
mixed_data = pd.DataFrame({
    'id': ['1', '2', '3', '4', '5'],
    'price': ['100.50', '200', '350.75', 'N/A', '500'],
    'quantity': [1.0, 2.0, 3.0, 4.0, 5.0],
    'date_str': ['2024-01-01', '2024/02/15', 'Mar 20, 2024', '20-04-2024', '2024.05.01'],
    'status': ['active', 'inactive', 'active', 'active', 'inactive']
})

print("=== Original Data Types ===")
print(mixed_data.dtypes)
print(f"\nMemory usage: {mixed_data.memory_usage(deep=True).sum()} bytes")

In [ ]:
def convert_types(df: pd.DataFrame) -> pd.DataFrame:
    """Convert columns to appropriate data types."""
    converted = df.copy()
    
    # String to integer
    converted['id'] = pd.to_numeric(converted['id'], errors='coerce').astype('Int64')
    
    # String to float (handle N/A)
    converted['price'] = pd.to_numeric(converted['price'], errors='coerce')
    
    # Float to integer
    converted['quantity'] = converted['quantity'].astype('Int64')
    
    # String to datetime (handle multiple formats)
    converted['date'] = pd.to_datetime(converted['date_str'], errors='coerce', dayfirst=False)
    converted = converted.drop('date_str', axis=1)
    
    # String to category (memory optimization)
    converted['status'] = converted['status'].astype('category')
    
    return converted

converted_data = convert_types(mixed_data)
print("=== Converted Data Types ===")
print(converted_data.dtypes)
print(f"\nMemory usage: {converted_data.memory_usage(deep=True).sum()} bytes")
print("\n", converted_data)

---
## 5. Aggregation

**Aggregation** summarizes detailed data into higher-level metrics.

### Common Aggregation Functions

```
┌────────────────────────────────────────────────────────┐
│ Numeric: sum, mean, median, min, max, std, var, count │
│ String:  first, last, count, nunique                  │
│ Date:    min, max, count                              │
│ Custom:  lambda x: your_function(x)                   │
└────────────────────────────────────────────────────────┘
```

In [ ]:
# Detailed sales data
sales = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=20, freq='D').repeat(3),
    'region': ['North', 'South', 'West'] * 20,
    'product': np.random.choice(['A', 'B', 'C'], 60),
    'quantity': np.random.randint(1, 100, 60),
    'revenue': np.random.uniform(100, 1000, 60).round(2)
})

print("=== Sample Sales Data (first 10 rows) ===")
print(sales.head(10))

In [ ]:
# Multiple aggregation examples

# 1. Simple group-by aggregation
region_summary = sales.groupby('region').agg({
    'quantity': 'sum',
    'revenue': ['sum', 'mean', 'count']
}).round(2)
region_summary.columns = ['total_qty', 'total_revenue', 'avg_revenue', 'num_transactions']
print("=== By Region ===")
print(region_summary)

# 2. Multi-level aggregation
pivot_summary = sales.pivot_table(
    values='revenue',
    index='region',
    columns='product',
    aggfunc=['sum', 'count'],
    fill_value=0
).round(2)
print("\n=== Pivot Table (Region x Product) ===")
print(pivot_summary)

In [ ]:
# 3. Rolling/Window aggregations
daily_sales = sales.groupby('date')['revenue'].sum().reset_index()
daily_sales['rolling_7d_avg'] = daily_sales['revenue'].rolling(window=7).mean()
daily_sales['cumulative_revenue'] = daily_sales['revenue'].cumsum()
daily_sales['pct_change'] = daily_sales['revenue'].pct_change() * 100

print("=== Time-based Aggregations ===")
print(daily_sales.round(2).tail(10))

---
## 6. Joining/Merging

**Joining** combines data from multiple sources based on common keys.

### Join Types

```
      LEFT JOIN                 INNER JOIN              FULL OUTER JOIN
    ┌─────┬─────┐             ┌─────┬─────┐             ┌─────┬─────┐
    │█████│     │             │     │     │             │█████│█████│
    │█████│████ │             │ ████│████ │             │█████│█████│
    │█████│     │             │     │     │             │█████│█████│
    └─────┴─────┘             └─────┴─────┘             └─────┴─────┘
    All from left +           Only matching             All from both
    matching right            records                   sides
```

In [ ]:
# Sample tables
orders = pd.DataFrame({
    'order_id': [1, 2, 3, 4, 5],
    'customer_id': [101, 102, 103, 104, 105],
    'amount': [500, 750, 200, 1200, 350]
})

customers_dim = pd.DataFrame({
    'customer_id': [101, 102, 103, 106],  # Note: 104, 105 missing; 106 extra
    'name': ['Alice', 'Bob', 'Charlie', 'Frank'],
    'segment': ['Enterprise', 'SMB', 'Enterprise', 'SMB']
})

print("=== Orders ===")
print(orders)
print("\n=== Customers ===")
print(customers_dim)

In [ ]:
# Different join types
print("=== INNER JOIN ===")
inner = orders.merge(customers_dim, on='customer_id', how='inner')
print(inner)

print("\n=== LEFT JOIN ===")
left = orders.merge(customers_dim, on='customer_id', how='left')
print(left)

print("\n=== OUTER JOIN ===")
outer = orders.merge(customers_dim, on='customer_id', how='outer')
print(outer)

In [ ]:
# Advanced: Multiple key join and self-join
order_items = pd.DataFrame({
    'order_id': [1, 1, 2, 2, 3],
    'product_id': ['P1', 'P2', 'P1', 'P3', 'P2'],
    'quantity': [2, 1, 3, 2, 1]
})

prices = pd.DataFrame({
    'product_id': ['P1', 'P2', 'P3'],
    'price': [100, 250, 75]
})

# Multi-step join
order_details = (
    order_items
    .merge(prices, on='product_id')
    .assign(line_total=lambda x: x['quantity'] * x['price'])
    .merge(orders[['order_id', 'customer_id']], on='order_id')
)

print("=== Order Details with Prices ===")
print(order_details)

---
## 7. SQL vs Python Transformations

Most transformations can be done in either SQL or Python. Here's a comparison:

| Operation | SQL | Python (pandas) |
|-----------|-----|------------------|
| Filter | `WHERE condition` | `df[df['col'] > value]` |
| Select | `SELECT col1, col2` | `df[['col1', 'col2']]` |
| Aggregate | `GROUP BY ... SUM()` | `df.groupby().agg()` |
| Join | `JOIN ... ON` | `df.merge()` |
| Sort | `ORDER BY` | `df.sort_values()` |
| Window | `OVER (PARTITION BY)` | `df.groupby().transform()` |

In [ ]:
# SQL-equivalent transformations in pandas

# Sample data
employees = pd.DataFrame({
    'emp_id': range(1, 11),
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve', 'Frank', 'Grace', 'Henry', 'Ivy', 'Jack'],
    'department': ['Sales', 'Sales', 'Engineering', 'Engineering', 'Sales', 'HR', 'Engineering', 'HR', 'Sales', 'Engineering'],
    'salary': [70000, 65000, 95000, 105000, 72000, 55000, 88000, 58000, 68000, 92000],
    'hire_date': pd.to_datetime(['2020-01-15', '2019-06-01', '2021-03-20', '2018-11-10', '2022-02-28',
                                  '2020-08-05', '2021-07-12', '2019-04-22', '2023-01-10', '2022-09-01'])
})

print("=== Employee Data ===")
print(employees)

In [ ]:
# SQL: SELECT * FROM employees WHERE salary > 70000 ORDER BY salary DESC
high_earners = (
    employees[employees['salary'] > 70000]
    .sort_values('salary', ascending=False)
)
print("=== High Earners (salary > 70k) ===")
print(high_earners)

# SQL: SELECT department, COUNT(*), AVG(salary) FROM employees GROUP BY department
dept_stats = (
    employees
    .groupby('department')
    .agg(
        employee_count=('emp_id', 'count'),
        avg_salary=('salary', 'mean'),
        total_salary=('salary', 'sum')
    )
    .round(2)
)
print("\n=== Department Statistics ===")
print(dept_stats)

In [ ]:
# SQL: Window function equivalent
# SELECT *, 
#   AVG(salary) OVER (PARTITION BY department) as dept_avg,
#   RANK() OVER (PARTITION BY department ORDER BY salary DESC) as salary_rank
# FROM employees

employees_with_windows = employees.copy()

# Department average (window function)
employees_with_windows['dept_avg_salary'] = (
    employees_with_windows
    .groupby('department')['salary']
    .transform('mean')
)

# Salary rank within department
employees_with_windows['salary_rank'] = (
    employees_with_windows
    .groupby('department')['salary']
    .rank(ascending=False, method='dense')
    .astype(int)
)

# Salary vs department average
employees_with_windows['vs_dept_avg'] = (
    (employees_with_windows['salary'] / employees_with_windows['dept_avg_salary'] - 1) * 100
).round(1)

print("=== Employees with Window Calculations ===")
print(employees_with_windows[['name', 'department', 'salary', 'dept_avg_salary', 'salary_rank', 'vs_dept_avg']])

### SQL Transformation Example

```sql
-- Equivalent SQL for the above transformations

-- Cleansing: Remove duplicates
WITH deduplicated AS (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY created_at) as rn
    FROM raw_customers
)
SELECT * FROM deduplicated WHERE rn = 1;

-- Aggregation with window functions
SELECT 
    emp_id,
    name,
    department,
    salary,
    AVG(salary) OVER (PARTITION BY department) as dept_avg_salary,
    RANK() OVER (PARTITION BY department ORDER BY salary DESC) as salary_rank,
    ROUND((salary / AVG(salary) OVER (PARTITION BY department) - 1) * 100, 1) as vs_dept_avg
FROM employees;

-- Joining multiple tables
SELECT 
    o.order_id,
    c.customer_name,
    p.product_name,
    oi.quantity * p.price as line_total
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
INNER JOIN order_items oi ON o.order_id = oi.order_id
INNER JOIN products p ON oi.product_id = p.product_id;
```

---
## 8. Transformation Pipeline Pattern

A reusable pattern for building maintainable transformation pipelines:

In [ ]:
from typing import Callable, List
from functools import reduce

class TransformationPipeline:
    """A composable transformation pipeline for DataFrames."""
    
    def __init__(self):
        self.steps: List[Callable[[pd.DataFrame], pd.DataFrame]] = []
    
    def add_step(self, func: Callable[[pd.DataFrame], pd.DataFrame], name: str = None):
        """Add a transformation step to the pipeline."""
        func.__name__ = name or func.__name__
        self.steps.append(func)
        return self
    
    def run(self, df: pd.DataFrame, verbose: bool = False) -> pd.DataFrame:
        """Execute all transformation steps in order."""
        result = df.copy()
        for step in self.steps:
            if verbose:
                print(f"Running: {step.__name__}")
            result = step(result)
        return result

# Define individual transformation functions
def clean_nulls(df):
    return df.fillna({'name': 'Unknown', 'amount': 0})

def standardize_text(df):
    df = df.copy()
    df['name'] = df['name'].str.strip().str.title()
    return df

def add_derived_fields(df):
    df = df.copy()
    df['amount_category'] = pd.cut(df['amount'], bins=[0, 100, 500, float('inf')], labels=['Low', 'Medium', 'High'])
    return df

# Build and run pipeline
pipeline = (
    TransformationPipeline()
    .add_step(clean_nulls, 'Clean Nulls')
    .add_step(standardize_text, 'Standardize Text')
    .add_step(add_derived_fields, 'Add Categories')
)

# Test data
test_data = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['  alice  ', None, 'BOB'],
    'amount': [50, 250, None]
})

result = pipeline.run(test_data, verbose=True)
print("\n=== Pipeline Result ===")
print(result)

---
## 📌 Key Takeaways

### Transformation Categories

| Category | Purpose | Key Operations |
|----------|---------|----------------|
| **Cleansing** | Fix data quality issues | Null handling, deduplication, outlier treatment |
| **Normalization** | Standardize scales/formats | Min-max, z-score, log transform |
| **Enrichment** | Add context/derived data | Lookups, calculated fields, geocoding |
| **Type Conversion** | Ensure correct data types | Parse dates, numeric conversion, categorization |
| **Aggregation** | Summarize data | Group by, pivot, window functions |
| **Joining** | Combine data sources | Inner, left, outer, cross joins |

### Best Practices

1. **Idempotency**: Transformations should produce the same result when run multiple times
2. **Immutability**: Create new DataFrames instead of modifying in place
3. **Composability**: Build small, focused functions that can be chained
4. **Testing**: Validate transformations with unit tests and data quality checks
5. **Documentation**: Document transformation logic and business rules

### SQL vs Python Decision

| Use SQL When | Use Python When |
|--------------|------------------|
| Data is already in database | Complex logic/loops required |
| Simple aggregations/joins | ML preprocessing needed |
| Performance-critical queries | Prototyping/exploration |
| Team SQL proficiency is high | External API calls needed |